In [2]:
from functions import get_points, write_to_laz, find_plane, find_clusters,kmeans_clusters

import pandas as pd
from pyntcloud import PyntCloud
import numpy as np
import pdal
import shapely.wkt
from numpy.lib import recfunctions as rfn

import numpy as np
import pandas as pd
import matplotlib
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from pyntcloud import PyntCloud
from sklearn.cluster import DBSCAN

from scipy.spatial import cKDTree

In [3]:
READ_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "readers.ept",
            "filename": "{path}",
            "bounds": "{bounds}"
        }},
        {{
            "type":"filters.crop",
            "polygon":"{wkt}"
        }}
    ]
}}
"""

WRITE_PIPELINE = """
{{
    "pipeline": [
        {{
            "type": "writers.las",
            "filename": "{path}",
            "extra_dims": "all"
        }}
    ]
}}
"""


def get_points(ept_path, bounds, wkt):
    pipeline = pdal.Pipeline(READ_PIPELINE.format(
        path=ept_path,
        bounds=bounds,
        wkt=wkt)
    )

    pipeline.validate()
    pipeline.execute()
    point_cloud = pipeline.arrays[0]

    return point_cloud


def write_to_laz(point_cloud, path):
    pipeline = pdal.Pipeline(
        WRITE_PIPELINE.format(path=path),
        arrays=[point_cloud]
    )
    pipeline.validate()
    pipeline.execute()


def hausdorff_distance(reference_cloud, compared_cloud):
    """

    Parameters
    ----------
    reference_cloud : (Mx3) array
        The X, Y, Z coordinates of points in the reference cloud
    compared_cloud : (Mx3) array
        The X, Y, Z coordinates of points in the compared cloud
    """
    tree = cKDTree(reference_cloud)
    distances, _ = tree.query(compared_cloud, k=1)
    return distances

def filter_distance(reference_cloud, compared_cloud, max_dist = 0.05):
    reference_array = np.array(reference_cloud[['X', 'Y', 'Z']].tolist())
    compared_array = np.array(compared_cloud[['X', 'Y', 'Z']].tolist())

    haus_distances = hausdorff_distance(reference_array, 
                                        compared_array)
    out_array = rfn.append_fields(compared_cloud[['X','Y','Z','Red','Green','Blue']], 
                                'Distance', 
                                haus_distances)
    filtered_out_array = out_array[out_array['Distance'] < max_dist]

    return filtered_out_array

In [4]:
test_area = [117813.6,487266.1,117942.1,487370.7]
xmin,ymin,xmax,ymax = test_area

path_a10_2018 = '/var/data/rws/data/2018/entwined/ept.json'
path_a10_2019 = '/var/data/rws/data/2019/amsterdam_entwined/ept.json'

# random area: 
# wkt = 'POLYGON((117813.6 487266.1,117813.6 487370.7,117942.1 487370.7,117942.1 487266.1,117813.6 487266.1))'

# small area: 
# wkt = 'POLYGON((117813 487266,117813 487270,117815 487270,117815 487266,117813 487266))' # small! 

# portal:
# wkt = 'POLYGON((118032.6 488782.8,118032.6 488793.5,118078.8 488793.5,118078.8 488782.8,118032.6 488782.8))' # portal

# portal zonder palen
# wkt = 'POLYGON((118037.0 488782.8,118037.0 488793.5,118072.0 488793.5,118072.0 488782.8,118037.0 488782.8))' # portal zonder palen

# brug
# wkt = 'POLYGON((117875.3 487304,117875.3 487329,117906.1 487329,117906.1 487304,117875.3 487304))' # brug

# stukje brug
wkt = 'POLYGON((117890.1 487311,117890.1 487312.5,117892 487312.5,117892 487311,117890.1 487311))' # brug


xmin,ymin,xmax,ymax = shapely.wkt.loads(wkt).bounds

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc18 = get_points(path_a10_2018, bounds, wkt)
print(f'loaded pc 2018')

bounds = f'([{xmin}, {xmax}], [{ymin}, {ymax}])'
pc19 = get_points(path_a10_2019, bounds, wkt)
print(f'loaded pc 2019')

loaded pc 2018
loaded pc 2019


In [4]:
# chris' params
min_size = 3
ransac_distance = 0.2
ransac_iterations = 1000
dbscan_distance = 1
dbscan_min_samples = 8

# my params
eps1 = 1.2
min_size1 = 8
eps2 = 0.8
min_size2 = 6

points_df = pd.DataFrame(
    np.array(pc1[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])


# points = np.array((x, y, z)).T
# points=pd.DataFrame(points,columns=['x','y','z'])
# roof = PyntCloud(points)
# plpy.warning('Roofpoints in: ',len(roof.points))

print('done')

NameError: name 'pc1' is not defined

In [7]:
filtered_2018 = filter_distance(pc18, pc19)
filtered_2019 = filter_distance(pc19, pc18)

points_2018 = pd.DataFrame(
    np.array(filtered_2018[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])

print('2018')

points_2019 = pd.DataFrame(
    np.array(filtered_2019[['X','Y','Z','Red','Green','Blue']].tolist()), 
    columns=['x', 'y', 'z','red','green','blue'])

print('2019')

In [118]:
np_hist_18 = np.histogram(points_2018['z'], bins=2)
np_hist_19 = np.histogram(points_2019['z'], bins=2)

In [ ]:
km_19 = kmeans_clusters(points_2019, 2 )
km_18 = kmeans_clusters(points_2018, 2 )

In [37]:
cluster_19 = rfn.append_fields(filtered_2018[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              km18['cluster'],
                              usemask=False
                             )

cluster_19 = rfn.append_fields(filtered_2019[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              km19['cluster'],
                              usemask=False
                             )

(array([0, 1], dtype=int32), array([411711,  53425]))

In [40]:
# arr = np.where(np.array(points_2019['z'])>= np_hist_19[1][1], 1, 0) 

arr = km['cluster']

out_array = rfn.append_fields(filtered_2019[['X','Y','Z','Red','Green','Blue']], 
                              'Classification', 
                              arr,
                              usemask=False
                             )

write_to_laz(out_array, '/var/data/rws/data/output/cluster_test/tst_19_1.laz')
print('wrote')

wrote


In [106]:
print(pc19.shape,
points_2019.shape)

(588813,) (465136, 6)


In [101]:
find_clusters(points_df, 
              eps= 1, 
              min_samples = 5000)

print(np.unique(np.array(points_df['cluster']), 
                return_counts=True))

print('found planes ?')

RuntimeError: Unable to fetch data and convert as requested: Classification:uint16_t(16959) -> unsigned char

In [ ]:
np.unique(arr, return_counts=True)

In [ ]:
bounds

In [ ]:
for i in np.array(ins['is_plane']):
    
    if i != 1:
        print(i)

In [72]:
from .. import hausorf_dist

ValueError: attempted relative import beyond top-level package